In [1]:
import os, re

import discovery as ds

import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as pp

import jax
import jax.numpy as jnp

In [314]:
import imp

imp.reload(ds.matrix)
imp.reload(ds.signals)
imp.reload(ds.likelihood)
imp.reload(ds.prior)
imp.reload(ds.pulsar)
imp.reload(ds)

<module 'discovery' from '/Users/vallis/Documents/discovery/src/discovery/__init__.py'>

In [315]:
ds.config('numpy')

Read pulsar data

In [2]:
psrfiles = !ls ../data
psrs = [ds.Pulsar.read_feather(f'../data/{psrfile}') for psrfile in psrfiles[:10]
        if '-J' in psrfile or '-B' in psrfile]

Choose the simulation

In [3]:
rndseed = 42

Choose the outpsr

In [4]:
outpsr = 0
outpsrname = psrs[outpsr].name

How many predictive samples?

In [5]:
nsamples = 1

Get the chains

In [6]:
model2a = pd.read_feather('../data/NG15yr-m2a-chain.feather')
model3a = pd.read_feather('../data/NG15yr-m3a-chain.feather')

Simulate model 2a

In [321]:
Tspan = ds.getspan(psrs)

cbl = ds.GlobalLikelihood([ds.PulsarLikelihood([psr.residuals,
                                                ds.makenoise_measurement(psr, psr.noisedict),
                                                ds.makegp_ecorr(psr, psr.noisedict),
                                                ds.makegp_timing(psr, variance=1e-14),
                                                ds.makegp_fourier(psr, ds.powerlaw, 30, T=Tspan, name='red_noise'),
                                                ds.makegp_fourier(psr, ds.powerlaw, 14, T=Tspan, name='gw', common=['gw_gamma', 'gw_log10_A'])])
                           for ipsr, psr in enumerate(psrs)])

sample = cbl.sample

In [322]:
p0 = model2a.sample(1).iloc[0].to_dict()

In [323]:
p0['gw_gamma'] = 4.33
p0['gw_log10_A'] = -14.5

In [327]:
key = ds.matrix.jnpkey(rndseed)
key, ysim = sample(key, p0)

In [155]:
str_log10_A = re.sub('\.','p',re.sub('\-','m',str(p0['gw_log10_A'])))
dirname = f'sim_model2a_{str_log10_A}_{rndseed}'

In [164]:
os.makedirs(dirname, exist_ok=True) 

In [181]:
for psr, ypsr in zip(psrs, ysim):
    psr.residuals = np.asarray(ypsr)
    psr.save_feather(os.path.join(dirname, f'{psr.name}.feather'), noisedict = psr.noisedict)

In [ ]:
del cbl, sample

Sample model 3a with vmap

In [7]:
Tspan = ds.getspan(psrs)

m3a = ds.GlobalLikelihood((ds.PulsarLikelihood([psr.residuals,
                                                ds.makenoise_measurement(psr, psr.noisedict),
                                                ds.makegp_ecorr(psr, psr.noisedict),
                                                ds.makegp_timing(psr, variance=1e-14),
                                                ds.makegp_fourier(psr, ds.powerlaw, 30, T=Tspan, name='red_noise')])
                           for ipsr, psr in enumerate(psrs)),
                          ds.makegp_fourier_global(psrs, ds.powerlaw, ds.hd_orf, 14, T=Tspan, name='gw'))

logl = m3a.logL
jlogl = jax.jit(logl)

In [228]:
imp.reload(ds.matrix)
imp.reload(ds.signals)
imp.reload(ds.likelihood)
imp.reload(ds)

<module 'discovery' from '/Users/vallis/Documents/discovery/src/discovery/__init__.py'>

Conditional probability for RN

In [229]:
Tspan = ds.getspan(psrs)

c2a = ds.GlobalLikelihood((ds.PulsarLikelihood([psr.residuals,
                                                ds.makenoise_measurement(psr, psr.noisedict),
                                                ds.makegp_ecorr(psr, psr.noisedict),
                                                ds.makegp_timing(psr, variance=1e-14)])
                           for ipsr, psr in enumerate(psrs)),
                          [ds.makegp_fourier_allpsr(psrs, ds.powerlaw, 30, T=Tspan, name='red_noise'),
                           ds.makegp_fourier_allpsr(psrs, ds.powerlaw, 14, T=Tspan, name='gw', common=['gw_gamma','gw_log10_A'])])

cond = c2a.conditional

In [234]:
p0 = ds.sample_uniform(cond.params)

In [ ]:
cond(p0)

In [241]:
Tspan = ds.getspan(psrs)

c2a = ds.GlobalLikelihood((ds.PulsarLikelihood([psr.residuals,
                                                ds.makenoise_measurement(psr, psr.noisedict),
                                                ds.makegp_ecorr(psr, psr.noisedict),
                                                ds.makegp_timing(psr, variance=1e-14)])
                           for ipsr, psr in enumerate(psrs)),
                          [ds.makegp_fourier_allpsr(psrs, ds.powerlaw, 30, T=Tspan, name='red_noise'),
                           ds.makegp_fourier_global(psrs, ds.powerlaw, ds.hd_orf, 14, T=Tspan, name='gw')])

cond = c3a.conditional

In [242]:
p0 = ds.sample_uniform(cond.params)

In [243]:
cond(p0)

TypeError: add got incompatible shapes for broadcasting: (281, 880), (880, 880).

In [206]:
jnp.isnan(c3a.globalgp.Phi_inv(p0)[0])

Array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]], dtype=bool)

In [192]:
cond(ds.sample_uniform(cond.params))

(Array([nan, nan, nan, ..., nan, nan, nan], dtype=float64),
 (Array([[nan, nan, nan, ..., nan, nan, nan],
         [ 0., nan, nan, ..., nan, nan, nan],
         [ 0.,  0., nan, ..., nan, nan, nan],
         ...,
         [ 0.,  0.,  0., ..., nan, nan, nan],
         [ 0.,  0.,  0., ...,  0., nan, nan],
         [ 0.,  0.,  0., ...,  0.,  0., nan]], dtype=float64),
  False))

In [175]:
p0 = ds.sample_uniform(cond.params)

In [161]:
rn = ds.makegp_fourier_allpsr(psrs, ds.powerlaw, 30, T=Tspan, name='red_noise')

In [162]:
gw = ds.makegp_fourier_global(psrs, ds.powerlaw, ds.hd_orf, 14, T=Tspan, name='gw')

In [163]:
cp = ds.matrix.CompoundGlobalGP([rn, gw])

In [164]:
cp.Phi_inv(ds.sample_uniform(cp.Phi_inv.params))

(Array([[2.23171441e+10, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 2.23171441e+10, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 2.82818537e+11, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         1.33611974e+13, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 2.21487254e+13, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 2.21487254e+13]], dtype=float64),
 Array(-2042.83589672, dtype=float64))

In [71]:
[np.hstack(F) for F in zip(*[rn.Fs, gw.Fs, rn.Fs])][0].shape

(7753, 148)

In [67]:
gw.Phi

In [58]:
gw.Fs[0].shape

(7753, 28)

In [63]:
np.hstack([rn.Fs[0], gw.Fs[0]]).shape

(7753, 88)

In [8]:
p0 = ds.sample_uniform(logl.params, n=10)

In [21]:
p0 = {par: ds.matrix.jnparray(val) for par, val in ds.sample_uniform(logl.params, n=2).items()}

In [14]:
vlogl = jax.vmap(logl)

In [16]:
jvlogl = jax.vmap(jlogl)

In [22]:
jvlogl(p0)

Array([1183333.61873163, 1187209.95184411], dtype=float64)

Model-3a conditional predictive 

In [330]:
Tspan = ds.getspan(psrs)

outnoise = ds.makenoise_measurement_simple(psrs[outpsr], {f'{outpsrname}_efac': 1.0,
                                                          f'{outpsrname}_log10_t2equad': 0.0})

gbl = ds.GlobalLikelihood([ds.PulsarLikelihood([psr.residuals,
                                                outnoise if ipsr == outpsr else ds.makenoise_measurement(psr, psr.noisedict),
                                                ds.makegp_ecorr(psr, psr.noisedict),
                                                ds.makegp_timing(psr, variance=1e-14),
                                                ds.makegp_fourier(psr, ds.powerlaw, 30, T=Tspan, name='red_noise')])
                           for ipsr, psr in enumerate(psrs)],
                          ds.makegp_fourier_global(psrs, ds.powerlaw, ds.hd_orf, 14, T=Tspan, name='gw'))

cond = gbl.conditional

In [13]:
# check that we have all the parameters
# [p for p in cond.params if p not in chain.columns]

outpsr model3a likelihood 

In [293]:
psr = psrs[outpsr]
spl3 = ds.PulsarLikelihood([psr.residuals,
                            ds.makenoise_measurement(psr, psr.noisedict),
                            ds.makegp_ecorr(psr, psr.noisedict),
                            ds.makegp_timing(psr, variance=1e-14),
                            ds.makegp_fourier(psr, ds.powerlaw, 30, T=Tspan, name='red_noise'),
                            ds.makegp_fourier_delay(psr, 14, T=Tspan, name='gw'),
                            ds.makegp_fourier_variance(psr, 14, T=Tspan, name='gw')
                            ])

logl3 = spl3.logL
jlogl3 = jax.jit(logl3)

In [294]:
outpsrrange  = slice(outpsr*28, (outpsr+1)*28)

outpsrmean = f'{outpsrname}_gw_mean(28)'
outpsrvar  = f'{outpsrname}_gw_variance(28,28)'

outpsrgamma  = f'{outpsrname}_red_noise_gamma'
outpsrlog10A = f'{outpsrname}_red_noise_log10_A'

In [298]:
def makeppl(logl, chain, conditional=True, accelerate=True):
    index, records = [], []
    for i, row in chain.sample(nsamples).iterrows():
        par = row.to_dict()

        if conditional:    
            cm, cl = cond(par)
            
            par[outpsrmean] = cm[outrange]
            par[outpsrvar] = ds.matrix.jsp.linalg.cho_solve(cl, jnp.identity(len(cm))[:, outrange])[outrange, :]

            if accelerate:
                # could cast this in PulsarLikelihood
                ym = np.array(ds.makegp_fourier_delay(psr, 14, T=Tspan, name='gw')(par))
                spl3 = ds.PulsarLikelihood([psr.residuals - ym,
                                            ds.makenoise_measurement(psr, psr.noisedict),
                                            ds.makegp_ecorr(psr, psr.noisedict),
                                            ds.makegp_timing(psr, variance=1e-14),
                                            ds.makegp_fourier(psr, ds.powerlaw, 30, T=Tspan, name='red_noise'),
                                            ds.makegp_fourier_variance(psr, 14, T=Tspan, name='gw', noisedict=par)
                                            ])
                
                logl = jax.jit(spl3.logL)
        
        def ufunc(gamma, log10_A):
            return float(logl({**par, outpsrgamma: gamma, outpsrlog10A: log10_A}))
    
        norm = ufunc(4.0, -14.0)
        def nfunc(gamma, log10_A):
            return np.exp(ufunc(gamma, log10_A) - norm)
    
        index.append(i)
        records.append({'gw_gamma': par.get('gw_gamma', np.nan),
                        'gw_log10_A': par['gw_log10_A'],
                                              # note func(y, x), with limits given for x, y
                        'logl': norm + np.log(sp.integrate.dblquad(nfunc, -20, -11, 0, 7)[0])})
    
    return pd.DataFrame(records, index=index)

In [301]:
%time df = makeppl(logl3, model3a, conditional=True, accelerate=True)

CPU times: user 2.94 s, sys: 33.6 ms, total: 2.97 s
Wall time: 1.69 s


In [289]:
%time df = makeppl(logl3, model3a, conditional=True, accelerate=True)

CPU times: user 2.21 s, sys: 36.5 ms, total: 2.25 s
Wall time: 1.23 s


In [ ]:
%time df = makeppl(jlogl3, model3a, conditional=True)

In [ ]:
df.to_feather(f'{outpsrname}_ppl_model3a.feather')

In [ ]:
del spl3, logl3, jlogl3

outpsr model2a likelihood 

In [210]:
spl2 = ds.PulsarLikelihood([psr.residuals,
                            ds.makenoise_measurement(psr, psr.noisedict),
                            ds.makegp_ecorr(psr, psr.noisedict),
                            ds.makegp_timing(psr, variance=1e-14),
                            ds.makegp_fourier(psr, ds.powerlaw, 30, T=Tspan, name='red_noise'),
                            ds.makegp_fourier(psr, ds.powerlaw, 14, T=Tspan, name='gw', common=['gw_log10_A', 'gw_gamma'])
                            ])

logl2 = spl2.logL
jlogl2 = jax.jit(logl2)

In [211]:
%time df = makeppl(logl2, model2a, conditional=False)

CPU times: user 187 ms, sys: 336 ms, total: 523 ms
Wall time: 300 ms


In [ ]:
df.to_feather(f'{outpsrname}_ppl_model2a.feather')